# 9. Image adjustment: transforming image content

## 9.1. Color manipulation

**9.1. 色彩处理**

大多数操作颜色通道的函数都可以在子模块 `skimage.color` 中找到。

### 9.1.1. Conversion between color models

彩色图像可以使用不同的 [color spaces](https://en.wikipedia.org/wiki/Color_space) 来表示。最常见的 color spaces 之一是 [RGB space](https://en.wikipedia.org/wiki/RGB_color_model)，其中图像具有红色、绿色和蓝色通道。然而，其他颜色模型也被广泛使用，例如 [HSV color model](https://en.wikipedia.org/wiki/HSL_and_HSV)，其中色调、饱和度和明度是独立通道，或者用于打印的 [CMYK model](https://en.wikipedia.org/wiki/CMYK_color_model)。

`skimage.color` 提供了将图像与不同颜色空间相互转换的实用函数。Integer-type arrays 可以通过转换操作转换为 floating-point type：

In [3]:
# 明亮饱和的红色
import numpy as np
import skimage.color as color
red_pixel_rgb = np.array([[[255, 0, 0]]], dtype=np.uint8)
color.rgb2hsv(red_pixel_rgb)

array([[[0., 1., 1.]]])

In [4]:
# 较深的饱和蓝色
dark_blue_pixel_rgb = np.array([[[0, 0, 100]]], dtype=np.uint8)
color.rgb2hsv(dark_blue_pixel_rgb)

array([[[0.66666667, 1.        , 0.39215686]]])

In [5]:
# 饱和度较低的粉红色
pink_pixel_rgb = np.array([[[255, 100, 255]]], dtype=np.uint8)
color.rgb2hsv(pink_pixel_rgb)

array([[[0.83333333, 0.60784314, 1.        ]]])

### 9.1.2. Conversion from RGBA to RGB - Removing alpha channel through alpha blending

**9.1.2.从 RGBA 到 RGB 的转换 - 通过 alpha 混合删除 alpha 通道**

通过将 RGBA 图像与背景进行 alpha 混合将 RGBA 图像转换为 RGB 图像是通过 `rgba2rgb()` 实现的

In [10]:
from skimage.color import rgba2rgb
from skimage import data
img_rgba = data.logo()
img_rgb = rgba2rgb(img_rgba)

### 9.1.3. Conversion between color and gray values

**9.1.3. 颜色值和灰度值之间的转换**

RGB 图像转换为灰度图像是通过 `rgb2gray()` 实现的

In [11]:
from skimage.color import rgb2gray
from skimage import data
img = data.astronaut()
img_gray = rgb2gray(img)

`rgb2gray()` 使用颜色通道的非均匀加权，因为人眼对不同颜色的敏感度不同。因此，这样的权重确保了从 RGB 到灰度的[亮度保持](https://en.wikipedia.org/wiki/Grayscale#Converting_color_to_grayscale)：

In [12]:
red_pixel = np.array([[[255, 0, 0]]], dtype=np.uint8)
color.rgb2gray(red_pixel)

array([[0.2125]])

In [13]:
green_pixel = np.array([[[0, 255, 0]]], dtype=np.uint8)
color.rgb2gray(green_pixel)

array([[0.7154]])

使用 `gray2rgb()` 将灰度图像转换为 RGB 只需在三个颜色通道上复制灰度值。

### 9.1.4. Image inversion

**9.1.4. 图像反转**

反转图像也称为互补图像。对于二值图像，True 值变为 False，反之亦然。对于灰度图像，像素值由数据类型的最大值与实际值的差值代替。对于 RGB 图像，对每个通道进行相同的操作。该操作可以通过 `skimage.util.invert()` 来实现：

In [14]:
from skimage import util
img = data.camera()
inverted_img = util.invert(img)

### 9.1.5. Painting images with labels

**9.1.5. 用标签绘制图像**

`label2rgb()` 可用于使用标签数组在灰度图像上叠加颜色，以对要使用相同颜色表示的区域进行编码。

![](https://scikit-image.org/docs/stable/_images/sphx_glr_plot_join_segmentations_001.png)

## 9.2. Contrast and exposure

**9.2.对比度和曝光**

图像像素可以采用由图像的 `dtype` 确定的值（see [Image data types and what they mean](https://scikit-image.org/docs/stable/user_guide/data_types.html#data-types)），例如对于 `uint8` 图像为 0 到 255，对于浮点图像为 `[0, 1]`。然而，大多数图像要么具有较窄的值范围（因为对比度较差），要么大多数像素值集中在可访问值的子范围内。`skimage.exposure` 提供将强度值分布在更大范围内的函数。

第一类方法计算强度的非线性函数，其独立于特定图像的像素值。此类方法通常用于校正传感器或接收器（例如人眼）的已知非线性。一个众所周知的例子是 [Gamma correction](https://en.wikipedia.org/wiki/Gamma_correction)，在 `adjustment_gamma()` 中实现。

其他方法根据图像的直方图重新分配像素值。像素值的直方图是用 `skimage.exposure.histogram()` 计算的：

In [ ]:
image = np.array([[1, 3], [1, 1]])
exposure.histogram(image)

`histogram()` 返回每个 value bin 的像素数以及 bins 的中心。因此，`histogram()` 的行为与 `numpy.histogram()` 的行为略有不同，后者返回 bins 的边界。

最简单的对比度增强 `rescale_intensity()` 包括使用线性变换将像素值拉伸到整个允许的范围：

In [18]:
from skimage import exposure
text = data.text()
text.min(), text.max()

(10, 197)

In [19]:
better_contrast = exposure.rescale_intensity(text)
better_contrast.min(), better_contrast.max()

(0, 255)

即使图像使用整个值范围，有时值范围两端的权重也非常小。在这种情况下，使用图像的百分位数剪切像素值可以提高对比度（但会损失一些信息，因为此操作会使某些像素饱和）：

In [20]:
moon = data.moon()
v_min, v_max = np.percentile(moon, (0.2, 99.8))
v_min, v_max

(10.0, 186.0)

In [21]:
better_contrast = exposure.rescale_intensity(moon, in_range=(v_min, v_max))

函数 `equalize_hist()` 将像素值的累积分布函数 (cdf) 映射到线性 cdf，确保值范围的所有部分在图像中均等地表示。结果，对比度较差的大区域的细节得到增强。作为进一步的改进，可以使用 `equalize_adapthist()` 在图像的子区域中执行直方图均衡，以便校正图像上的曝光梯度。请参阅直方图均衡示例。

![](https://scikit-image.org/docs/stable/_images/sphx_glr_plot_equalize_001.png)